# Testing `run_symmetrized_readout`

The purpose of this notebook is to help in the code review of `run_symmetrized_readout`. 

The noise needed to demonstrate the functionarlity is hardcoded into the `api._quantum_computer`. We will need to remove it before merging this branch.

In [1]:
from math import pi
from typing import List
import itertools
import numpy as np

from pyquil import Program, get_qc
from pyquil.api import QuantumComputer
from pyquil.gates import *
from forest.benchmarking.utils import bitstring_prep

**get a quantum computers, because we need one**

In [2]:
from forest.benchmarking.compilation import basic_compile
from pyquil.api._qac import AbstractCompiler

# we need to do this otherwise everything gets complied away
class DummyCompiler(AbstractCompiler):
    def get_version_info(self):
        return {}

    def quil_to_native_quil(self, program: Program):
        return basic_compile(program)

    def native_quil_to_executable(self, nq_program: Program):
        return nq_program

qc = get_qc("8q-qvm")
qc.compiler = DummyCompiler()

**define some two qubit asymmetric noise operators**

Note these operators are not used below. Again they are hardcoded into `api._quantum_computer`. I put them here so you can see the kind of noise that the new methods symmetrize.

In [3]:
def two_qubit_bit_flip_operators(p00,p01,p10,p11):
    """
    Return a special case of a two qubit asymmetric bit flip kraus operators.
    
    Suppose we prepare a two qubit state |i,j> = |i>\otimes|j> for i,j \in {0,1}.
    
    Then pij := Pr(measured=ij|prepared=ij). So if pij = 1 no flip happens.
    
    For example consider p00 = 1-epsilon then a flip happens with probablity epsilon.
    The flip is symmetrically superposed over flipping to the states |0,1>, |1,0>, and |1,1>.
    The asymmetry comes from the fact that p00 does not have to be equal to p10 etc.
    
    :param p00: the probablity of |0,0> to remain in |0,0>
    :param p01: the probablity of |0,1> to remain in |0,1>
    :param p10: the probablity of |1,0> to remain in |1,0>
    :param p11: the probablity of |1,1> to remain in |1,1>
    :returns: a list of four Kraus operators.
    """
    p00e = 1.0 - p00
    p01e = 1.0 - p01
    p10e = 1.0 - p10
    p11e = 1.0 - p11
    kI = np.array([[np.sqrt(1-p00e), 0.0, 0.0, 0.0], [0.0, np.sqrt(1-p01e), 0.0, 0.0], [0.0, 0.0, np.sqrt(1-p10e), 0.0], [0.0, 0.0, 0.0, np.sqrt(1-p11e)]])
    k00 = np.sqrt(p00e/3) * ( _flip_matrix(0,1) + _flip_matrix(0,2) + _flip_matrix(0,3) )
    k01 = np.sqrt(p01e/3) * ( _flip_matrix(1,0) + _flip_matrix(1,2) + _flip_matrix(1,3) )
    k10 = np.sqrt(p10e/3) * ( _flip_matrix(2,0) + _flip_matrix(2,1) + _flip_matrix(2,3) )
    k11 = np.sqrt(p11e/3) * ( _flip_matrix(3,0) + _flip_matrix(3,1) + _flip_matrix(3,2) )
    return kI, k00, k01, k10, k11

def _flip_matrix(i,j,dim=4):
    mat = np.zeros((dim,dim))
    #mat.itemset((i,j),1)
    mat.itemset((j,i),1)
    return mat

def append_kraus_to_gate(kraus_ops, g):
    """
    Follow a gate `g` by a Kraus map described by `kraus_ops`.

    :param list kraus_ops: The Kraus operators.
    :param numpy.ndarray g: The unitary gate.
    :return: A list of transformed Kraus operators.
    """
    return [kj.dot(g) for kj in kraus_ops]

**If the noise model above** was compatible with the run methods then the 

In [4]:
nbits = 2
prepnames = []
prepprogs = []
binarray = []
for flip_array in itertools.product([0, 1], repeat=nbits):
    prepprogs.append(bitstring_prep(list(range(0,nbits)),flip_array))
    prepnames.append(''.join(str(x) for x in flip_array))
    binarray.append(list(flip_array))

In [5]:
from pyquil.quil import DefGate

II_mat = np.eye(4)
II_definition = DefGate("II", II_mat)
II = II_definition.get_constructor()
kraus_ops = two_qubit_bit_flip_operators(0.1,1,1,1)

p = Program(II_definition)

p.define_noisy_gate("II", [0, 1],  append_kraus_to_gate(kraus_ops, II_mat))
p += Program(II(0,1))

In [6]:
progRAM = p.copy()

prog = p.copy()
ro = prog.declare('ro', 'BIT', 2)
prog += MEASURE(0, ro[0])
prog += MEASURE(1, ro[1])
prog.wrap_in_numshots_loop(10)

# `run_and_measure`

the noise is only on qubits 0 and 1


In [7]:
qc.run_and_measure(progRAM,trials=10)

{0: array([1, 0, 0, 0, 1, 0, 1, 1, 0, 0]),
 1: array([0, 1, 1, 1, 1, 0, 0, 1, 1, 1]),
 2: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 3: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 4: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 5: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 6: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 7: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}

# `run` the noise models works :)

In [8]:
qc.run(prog)

array([[1, 0],
       [1, 1],
       [0, 1],
       [1, 1],
       [1, 1],
       [0, 1],
       [1, 1],
       [1, 1],
       [0, 1],
       [0, 1]])

# `run_symmetrized_readout_new` also works 

if you turn the compiler off

In [9]:
qc.run_symmetrized_readout(progRAM, trials=9, symm_type=2)

The number of trials was modified from 9 to 8. To be consistent with the number of trials required by the type of readout symmetrization chosen.


/Users/joshua/pyquil/pyquil/quil.py:183: UserWarning: Gate II has already been defined in this program
  .format(instruction.name))


array([[1, 0],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 0],
       [1, 0],
       [1, 0]])

# check the confusion matrix

Everything above here works. But for the code below you need to hack the noise models.

In [10]:
import pandas as pd

In [11]:
nbits = 2
prepnames = []
prepprogs = []
binarray = []
for flip_array in itertools.product([0, 1], repeat=nbits):
    prepprogs.append(bitstring_prep(list(range(0,nbits)),flip_array))
    prepnames.append(''.join(str(x) for x in flip_array))
    binarray.append(list(flip_array))

In [12]:
# kraus_ops = two_qubit_bit_flip_operators(1,0.7,1,1)

# # add noise
# for prog in prepprogs:
#     prog.inst(II_definition)
#     prog.define_noisy_gate("II", [0, 1],  append_kraus_to_gate(kraus_ops, II_mat))
#     prog.inst(II(0,1))

Note there is a really big hack here... I had to hard code the noise in. The above does note work.

The problem is that if you add the symmetrization using this method it happens after the noise which wont do anything

**trival / no symm**

In [13]:
data = []
num_shots = 6000

for prepname,program in zip(prepnames, prepprogs):
    answer = qc.run_symmetrized_readout(program, trials=num_shots, symm_type=0)
    counts=[]
    for cdx in binarray:
        counts.append(answer.tolist().count(cdx) / num_shots)

    dicty = dict(zip(['Prep']+[ 'Meas '+ name for name in prepnames], [prepname] + counts)) 
    data.append(dicty)

df = pd.DataFrame(data)
ndf = df.set_index('Prep',drop=True)
ndf

,Meas 00,Meas 01,Meas 10,Meas 11
Prep,,,,
00,0.693,0.102333,0.098333,0.106333
01,0.000,1.000000,0.000000,0.000000
10,0.000,0.000000,1.000000,0.000000
11,0.000,0.000000,0.000000,1.000000


**One design**

In this case does not help much

In [14]:
data = []
num_shots = 6000

for prepname,program in zip(prepnames, prepprogs):
    answer = qc.run_symmetrized_readout(program, trials=num_shots, symm_type=1)
    counts=[]
    for cdx in binarray:
        counts.append(answer.tolist().count(cdx) / num_shots)

    dicty = dict(zip(['Prep']+[ 'Meas ' + name for name in prepnames], [prepname] + counts)) 
    data.append(dicty)

df = pd.DataFrame(data)
ndf = df.set_index('Prep',drop=True)
ndf

,Meas 00,Meas 01,Meas 10,Meas 11
Prep,,,,
00,0.845333,0.055500,0.052167,0.047000
01,0.000000,1.000000,0.000000,0.000000
10,0.000000,0.000000,1.000000,0.000000
11,0.049333,0.054833,0.045000,0.850833


**two design**

:) it is working!!

In [15]:
data = []
num_shots = 6000

for prepname,program in zip(prepnames, prepprogs):
    answer = qc.run_symmetrized_readout(program, trials=num_shots, symm_type=2)
    counts=[]
    for cdx in binarray:
        counts.append(answer.tolist().count(cdx) / num_shots)

    dicty = dict(zip(['Prep']+[ 'Meas ' + name for name in prepnames], [prepname] + counts)) 
    data.append(dicty)

df = pd.DataFrame(data)
ndf = df.set_index('Prep',drop=True)
ndf

,Meas 00,Meas 01,Meas 10,Meas 11
Prep,,,,
00,0.927333,0.023833,0.022833,0.026000
01,0.024167,0.922667,0.025500,0.027667
10,0.028167,0.021500,0.926667,0.023667
11,0.024000,0.024667,0.022667,0.928667


**three design**

of course it works here

In [16]:
data = []
num_shots = 6000

for prepname,program in zip(prepnames, prepprogs):
    answer = qc.run_symmetrized_readout(program, trials=num_shots, symm_type=3)
    counts=[]
    for cdx in binarray:
        counts.append(answer.tolist().count(cdx) / num_shots)

    dicty = dict(zip(['Prep']+[ 'Meas ' + name for name in prepnames], [prepname] + counts)) 
    data.append(dicty)

df = pd.DataFrame(data)
ndf = df.set_index('Prep',drop=True)
ndf

,Meas 00,Meas 01,Meas 10,Meas 11
Prep,,,,
00,0.923000,0.026000,0.026667,0.024333
01,0.022500,0.928000,0.025000,0.024500
10,0.026333,0.026333,0.922333,0.025000
11,0.028833,0.026000,0.023000,0.922167


**exhaustive symm**

of course it works here

In [17]:
data = []
num_shots = 6000

for prepname,program in zip(prepnames, prepprogs):
    answer = qc.run_symmetrized_readout(program, trials=num_shots, symm_type=-1)
    counts=[]
    for cdx in binarray:
        counts.append(answer.tolist().count(cdx) / num_shots)

    dicty = dict(zip(['Prep']+[ 'Meas ' + name for name in prepnames], [prepname] + counts)) 
    data.append(dicty)

df = pd.DataFrame(data)
ndf = df.set_index('Prep',drop=True)
ndf

,Meas 00,Meas 01,Meas 10,Meas 11
Prep,,,,
00,0.926500,0.023167,0.025667,0.024667
01,0.022833,0.928667,0.024333,0.024167
10,0.025500,0.026667,0.924667,0.023167
11,0.024000,0.023167,0.029500,0.923333


**other things to play with**

In [18]:
from pyquil.api._quantum_computer import _symmetrization, _flip_array_to_prog

In [19]:
progy = _flip_array_to_prog((1,0,1,0,1,1),[0,1,2,3,4,5])
print(progy)

RX(pi) 0
RX(pi) 2
RX(pi) 4
RX(pi) 5



In [20]:
prog, flip_array = _symmetrization(Program(I(0),I(1)), [0,1], 1)

In [21]:
flip_array

[array([0, 0]), array([1, 1])]

In [22]:
print(prog)
[print(x) for x in prog]

[<pyquil.quil.Program object at 0xa2823ff60>, <pyquil.quil.Program object at 0xa2823fe48>]
DEFGATE II:
    1.0, 0, 0, 0
    0, 1.0, 0, 0
    0, 0, 1.0, 0
    0, 0, 0, 1.0

I 0
I 1
PRAGMA ADD-KRAUS II 0 1 "(0.8366600265340756 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0)"
PRAGMA ADD-KRAUS II 0 1 "(0.0 0.0 0.0 0.0 0.31622776601683794 0.0 0.0 0.0 0.31622776601683794 0.0 0.0 0.0 0.31622776601683794 0.0 0.0 0.0)"
PRAGMA ADD-KRAUS II 0 1 "(0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0)"
PRAGMA ADD-KRAUS II 0 1 "(0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0)"
PRAGMA ADD-KRAUS II 0 1 "(0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0)"
II 0 1

DEFGATE II:
    1.0, 0, 0, 0
    0, 1.0, 0, 0
    0, 0, 1.0, 0
    0, 0, 0, 1.0

I 0
I 1
RX(pi) 0
RX(pi) 1
PRAGMA ADD-KRAUS II 0 1 "(0.8366600265340756 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0)"
PRAGMA ADD-KRAUS II 0 1 "(0.0 0.0 0.0 0.0 0.31622776601683794 0.0 0.0 0.0 0

[None, None]

In [23]:
from pyquil.api._quantum_computer import (_construct_strength_three_orthogonal_array,
                                          _construct_strength_two_orthogonal_array)

In [24]:
_construct_strength_two_orthogonal_array(3)

array([[0, 0, 0],
       [1, 0, 1],
       [0, 1, 1],
       [1, 1, 0]])